In [1]:
import itertools
import pickle
import pandas as pd
import numpy as np
import torch
import torch_geometric.transforms
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from torch_geometric.nn import SAGEConv,to_hetero
from torch import Tensor
import tqdm
import torch.nn.functional as F

In [2]:
# data_trade = pd.read_excel("/home/QFRC/ZQS/Projects/Graph/data/图数据/图数值数据_label.xlsx", sheet_name="Sheet1")
# data_event = pd.read_excel("/home/QFRC/ZQS/Projects/Graph/data/图数据/图事件数据.xlsx", sheet_name="Sheet1")

data_trade = pd.read_excel("/home/QFRC/ZQS/Projects/Graph/data/图数据/图数值数据_label（spearman）.xlsx", sheet_name="Sheet1")
data_event = pd.read_excel("/home/QFRC/ZQS/Projects/Graph/data/图数据/图事件数据（spearman）2.0.xlsx", sheet_name="Sheet1")
data_spearman=pd.read_excel("/home/QFRC/ZQS/Projects/Graph/data/spearman_correlations.xlsx", sheet_name="Sheet1")
# print(type(data_trade))
print(data_trade)
# print(data_event)

     Unnamed: 0   浪潮信息 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0          交易日期   日开盘价       日最高价       日最低价       日收盘价    日个股交易股数   
1    2021-01-11  29.95      30.75      29.07      30.17  104342382   
2    2021-01-12   29.8      30.58      29.68       30.4   54960452   
3    2021-01-13  30.23      31.75      29.98       30.8   61509335   
4    2021-01-14   30.5      32.14      30.08      31.42   67303156   
..          ...    ...        ...        ...        ...        ...   
776  2024-03-25  40.71      42.18         40      40.26   92823641   
777  2024-03-26  39.61      42.26      39.61      40.53  111664559   
778  2024-03-27  40.03         42       39.4      40.32  123236968   
779  2024-03-28     41      44.35         41      44.35  177530162   
780  2024-03-29  44.49      44.79       41.6       42.9  203252699   

      Unnamed: 6 Unnamed: 7  Unnamed: 8 Unnamed: 9  ... Unnamed: 135  \
0    沪深港股通持股数量变动   沪深港通开通状态    融资净买入（元）   融券净卖出（股）  ...         日最低价   
1         156.1

In [3]:
# 事件处理
# 检查entity1，entity2是否属于股票节点类型，属于的话指向节点类型，不属于的话构建新节点类型（还是要先构建节点特征，不要用时序思维，而是要用并联思维）

# stocks_index_dic={"浪潮信息": 0, "中科曙光": 1, "科大讯飞": 2, "昆仑万维": 3, "中际旭创": 4, "景嘉微": 5, "比亚迪": 6, "长城汽车": 7, "长安汽车": 8, "宁德时代": 9, "天齐锂业": 10, "亿纬锂能": 11, "隆基绿能": 12, "晶澳科技": 13, "通威股份": 14, "TCL中环": 15, "阳光电源": 16, "正泰电器": 17}

# 和data_trade顺序相对应
stocks_index_dic={"浪潮信息": 0, "中科曙光": 1, "科大讯飞": 2, "昆仑万维": 3, "中际旭创": 4, "景嘉微": 5, "宁德时代": 6, "比亚迪": 7, "隆基绿能": 8, "亿纬锂能": 9, "通威股份": 10, "阳光电源": 11}
entity_add_dic1={}
entity_add_dic2={}
entity_add_dic3={}
entity_add_dic4={}

# def entity_process1(entity): 
#     # 查看是否属于股票节点
#     if entity in stocks_index_dic:
#         return stocks_index_dic[entity]
#     # 查看是否在已知的其他节点中
#     elif entity in entity_add_dic1:
#         return entity_add_dic1[entity]
#     # 添加到新的其他节点
#     else:
#         new_index = len(entity_add_dic1)
#         entity_add_dic1[entity]=new_index
#         return new_index
      
def entity_process1(entity): 
    # 查看是否属于股票节点
    if entity in stocks_index_dic:
        return stocks_index_dic[entity]
    # 查看是否在已知的其他节点中
    elif entity in entity_add_dic1:
        return entity_add_dic1[entity]
    # 添加到新的其他节点
    else:
        new_index = len(entity_add_dic1)
        entity_add_dic1[entity]=new_index
        return new_index    
 
def entity_process2(entity): 
    # 查看是否属于股票节点
    if entity in stocks_index_dic:
        return stocks_index_dic[entity]
    # 查看是否在已知的其他节点中
    elif entity in entity_add_dic2:
        return entity_add_dic2[entity]
    # 添加到新的其他节点
    else:
        new_index = len(entity_add_dic2)
        entity_add_dic2[entity]=new_index
        return new_index
    
def entity_process3(entity): 
    # 查看是否属于股票节点
    if entity in stocks_index_dic:
        return stocks_index_dic[entity]
    # 查看是否在已知的其他节点中
    elif entity in entity_add_dic3:
        return entity_add_dic3[entity]
    # 添加到新的其他节点
    else:
        new_index = len(entity_add_dic3)
        entity_add_dic3[entity]=new_index
        return new_index     
 
def entity_process4(entity): 
    # 查看是否属于股票节点
    if entity in stocks_index_dic:
        return stocks_index_dic[entity]
    # 查看是否在已知的其他节点中
    elif entity in entity_add_dic4:
        return entity_add_dic4[entity]
    # 添加到新的其他节点
    else:
        new_index = len(entity_add_dic4)
        entity_add_dic4[entity]=new_index
        return new_index    
    
entity_process1('奥利给')    

0

In [4]:
print(entity_add_dic1)
print(entity_process1('奥利给'))
str="奥利给"
if str in entity_add_dic1:
    print("奥利给")
else:
    print("奥利给2")

{'奥利给': 0}
0
奥利给


In [5]:
from transformers import BertModel, AutoTokenizer

model_path = '/home/QFRC/ZQS/Projects/Graph/Scripts_demo/NLP_models/Finbert'  # 你的模型路径

# 加载微调后的tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
# 加载微调后的模型
model = BertModel.from_pretrained(model_path, device_map="cuda:1", output_hidden_states=True)


In [6]:
# text编码测试
text1 = "ABCDEFG"  # 你的实际短语
text2 = "'江苏新能(603693)、隆基股份(601012)、通威股份(600438)、阳光电源(300274)、东方电缆(603606)'"

entity1 = "阳光电源"
entity2 = "逆变器"

with torch.no_grad():  # 关闭梯度计算
    outputs1 = model(**tokenizer(text1, return_tensors="pt").to("cuda:1"))
    outputs2 = model(**tokenizer(text2, return_tensors="pt").to("cuda:1"))

add_entity_last_layer = outputs1.hidden_states[-1]# 最后一层的输出，本身就已是张量
cls=add_entity_last_layer[:, 0, :]
relationship_last_layer = outputs2.hidden_states[-1]

# add_entity_last_layer = add_entity_last_layer.squeeze(0)  # 去掉无用的第一维
# relationship_last_layer = relationship_last_layer.squeeze(0)

# print(relationship_last_layer.shape)
# print(relationship_last_layer[:, -1, :].shape)
# print(relationship_last_layer[:, -1, :])
print(add_entity_last_layer.shape)
print(add_entity_last_layer)
print(cls.shape)
print(cls)

torch.Size([1, 6, 768])
tensor([[[ 0.1888, -0.4083,  0.5077,  ...,  0.2489, -0.5434, -0.6457],
         [ 0.1234, -0.1911, -0.8126,  ...,  0.1697, -0.1362,  0.0697],
         [ 0.2602, -0.9799,  0.5389,  ..., -0.3647, -0.5636, -0.5454],
         [-0.2390, -0.3979, -0.5498,  ..., -0.1381,  0.4279, -0.5545],
         [ 0.6862, -0.0138, -0.7177,  ...,  0.5496, -0.6156, -0.7364],
         [-0.1970, -0.2882,  0.0946,  ..., -0.7266,  0.3397, -0.4564]]],
       device='cuda:1')
torch.Size([1, 768])
tensor([[ 1.8876e-01, -4.0826e-01,  5.0769e-01,  3.2578e-01,  4.7953e-01,
         -7.0846e-01,  3.0010e-01,  1.7763e-02, -3.3916e-01, -8.2291e-02,
          6.1376e-01,  2.0679e-02,  1.4822e+00, -6.6652e-01, -8.0901e-01,
         -2.0017e-02,  8.5749e-01,  4.2456e-01,  3.7928e-01,  1.5818e+00,
         -7.0484e-01, -2.6492e-01, -4.2983e-01, -9.1816e-01,  7.5013e-01,
         -2.0063e-01, -9.1027e-01, -4.5572e-01,  8.9458e-03,  8.9310e-02,
          6.0522e-01, -6.9956e-01, -7.4183e-01,  4.2576e-01

In [7]:
def extract_last_layer_hidden(model, tokenizer, text, device='cuda:1'):
    """
    提取给定文本经过模型处理后最后一层的隐藏状态。
    
    参数:
    - model: 已加载的预训练模型。
    - tokenizer: 对应的分词器。
    - text: 需要处理的文本字符串。
    - device: 指定模型运行的设备，默认为 'cuda:1'。
    
    返回:
    - tensor: 处理后的最后一层隐藏状态张量。
    """
    # 将模型和数据转移到指定的设备上
    model = model.to(device)
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    # 关闭梯度计算，进行模型前向传播
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 获取最后一层的隐藏状态，并移除无用的维度
    last_layer_hidden_state = outputs.hidden_states[-1][:, 0, :].cpu()
    
    return last_layer_hidden_state

last_hidden_state = extract_last_layer_hidden(model, tokenizer, "我不明白")
print(last_hidden_state)
print(last_hidden_state.shape)

tensor([[-2.5722e-01,  5.1455e-01,  1.0347e+00,  2.9541e-01,  1.9830e-01,
         -8.6497e-01, -1.2650e-01,  1.3702e-02, -2.4924e-01,  1.0132e-01,
         -3.7447e-01,  5.6801e-01,  5.0084e-01, -1.1397e+00,  1.8955e-01,
         -6.7791e-01,  2.6501e-01,  5.2491e-01, -7.1945e-01,  5.0535e-01,
         -1.3303e+00,  8.0624e-02,  6.5307e-02, -1.2703e+00, -4.4784e-01,
          5.6249e-01, -1.9150e-01, -3.5233e-02,  3.2252e-01,  2.1788e-01,
         -3.9256e-01, -2.5833e-01, -7.6215e-01,  1.2376e-01, -8.8045e-01,
         -2.9461e-01,  3.5523e-01, -3.2248e-01,  1.2265e-01, -1.7271e-01,
          9.0833e-01, -4.7983e-01,  4.9570e-01,  1.3489e-01, -8.3262e-01,
         -1.3345e-01, -4.0449e-01,  3.4659e-01, -6.2601e-01, -1.0047e-01,
          3.3182e-01,  1.3576e+01, -3.1839e-01,  9.5921e-02, -1.6084e+00,
          4.7117e-01, -3.0645e-01,  1.7992e-01,  1.7539e-01,  1.7327e-01,
         -5.9553e-01, -6.8704e-01,  1.2663e-01,  1.6681e-01,  4.5929e-01,
         -1.5396e-01, -7.7569e-02,  1.

In [8]:
num_entity1=entity_process(entity1)
num_entity2=entity_process(entity2)

print(num_entity1)
print(num_entity2)

NameError: name 'entity_process' is not defined

In [9]:
# 设定时间窗口

names=locals() # 命名全局变量
window_size = 1


for m in range(1,900): # 创建序号1~1410的千张空的异构图
    names[f'graph_data{m}']=HeteroData()
    


In [10]:
data_trade_time=data_trade.iloc[1:, 0]
data_event_time=data_event['事件截止时间（数值）']

print(data_trade_time)  # 变成series
print(data_trade_time)  #也变成了series
# print(type(data_event))
event_unique_dates = data_event['事件截止时间（数值）'].unique()
print(event_unique_dates)

1      2021-01-11
2      2021-01-12
3      2021-01-13
4      2021-01-14
5      2021-01-15
          ...    
776    2024-03-25
777    2024-03-26
778    2024-03-27
779    2024-03-28
780    2024-03-29
Name: Unnamed: 0, Length: 780, dtype: object
1      2021-01-11
2      2021-01-12
3      2021-01-13
4      2021-01-14
5      2021-01-15
          ...    
776    2024-03-25
777    2024-03-26
778    2024-03-27
779    2024-03-28
780    2024-03-29
Name: Unnamed: 0, Length: 780, dtype: object
<DatetimeArray>
['2021-01-11 00:00:00', '2021-01-12 00:00:00', '2021-01-13 00:00:00',
 '2021-01-14 00:00:00', '2021-01-15 00:00:00', '2021-01-18 00:00:00',
 '2021-01-19 00:00:00', '2021-01-20 00:00:00', '2021-01-21 00:00:00',
 '2021-01-22 00:00:00',
 ...
 '2024-03-18 00:00:00', '2024-03-19 00:00:00', '2024-03-20 00:00:00',
 '2024-03-21 00:00:00', '2024-03-22 00:00:00', '2024-03-25 00:00:00',
 '2024-03-26 00:00:00', '2024-03-27 00:00:00', '2024-03-28 00:00:00',
 '2024-03-29 00:00:00']
Length: 780, dtype: datet

In [11]:
unique_dates = data_event['事件截止时间（数值）'].unique()

# 将这些唯一值转换为列表
event_unique_dates_list = list(unique_dates)

# 存储归档时间到本地
# with open('阳光电源事件归档时间.pkl', 'wb') as f:
#     pickle.dump(event_unique_dates_list, f)

print(event_unique_dates_list)


# 调用保存的pkl
# file_path = '/home/QFRC/ZQS/Projects/Graph/阳光电源归档时间.pkl'

# # 使用'rb'模式读取二进制文件
# with open(file_path, 'rb') as f:
#     # 使用pickle.load()加载文件内容，这将反序列化对象
#     data_loaded = pickle.load(f)
    
# data_loaded

[Timestamp('2021-01-11 00:00:00'), Timestamp('2021-01-12 00:00:00'), Timestamp('2021-01-13 00:00:00'), Timestamp('2021-01-14 00:00:00'), Timestamp('2021-01-15 00:00:00'), Timestamp('2021-01-18 00:00:00'), Timestamp('2021-01-19 00:00:00'), Timestamp('2021-01-20 00:00:00'), Timestamp('2021-01-21 00:00:00'), Timestamp('2021-01-22 00:00:00'), Timestamp('2021-01-25 00:00:00'), Timestamp('2021-01-26 00:00:00'), Timestamp('2021-01-27 00:00:00'), Timestamp('2021-01-28 00:00:00'), Timestamp('2021-01-29 00:00:00'), Timestamp('2021-02-01 00:00:00'), Timestamp('2021-02-02 00:00:00'), Timestamp('2021-02-03 00:00:00'), Timestamp('2021-02-04 00:00:00'), Timestamp('2021-02-05 00:00:00'), Timestamp('2021-02-08 00:00:00'), Timestamp('2021-02-09 00:00:00'), Timestamp('2021-02-10 00:00:00'), Timestamp('2021-02-18 00:00:00'), Timestamp('2021-02-19 00:00:00'), Timestamp('2021-02-22 00:00:00'), Timestamp('2021-02-23 00:00:00'), Timestamp('2021-02-24 00:00:00'), Timestamp('2021-02-25 00:00:00'), Timestamp('20

In [12]:
# 股票spearman边索引

num_nodes = 12
# 获取上三角矩阵的索引，不包括对角线
rows, cols = np.triu_indices(num_nodes, k=1)

# 创建边索引
spearman_edge_index = np.vstack([np.hstack([rows, cols]), np.hstack([cols, rows])])
print(spearman_edge_index)
print(spearman_edge_index.shape)
print(type(spearman_edge_index))

[[ 0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  2  2  2
   2  2  2  2  2  2  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  5  5  5
   5  5  5  6  6  6  6  6  7  7  7  7  8  8  8  9  9 10  1  2  3  4  5  6
   7  8  9 10 11  2  3  4  5  6  7  8  9 10 11  3  4  5  6  7  8  9 10 11
   4  5  6  7  8  9 10 11  5  6  7  8  9 10 11  6  7  8  9 10 11  7  8  9
  10 11  8  9 10 11  9 10 11 10 11 11]
 [ 1  2  3  4  5  6  7  8  9 10 11  2  3  4  5  6  7  8  9 10 11  3  4  5
   6  7  8  9 10 11  4  5  6  7  8  9 10 11  5  6  7  8  9 10 11  6  7  8
   9 10 11  7  8  9 10 11  8  9 10 11  9 10 11 10 11 11  0  0  0  0  0  0
   0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2
   3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  5  5  5  5  5  5  6  6  6
   6  6  7  7  7  7  8  8  8  9  9 10]]
(2, 132)
<class 'numpy.ndarray'>


In [13]:
# print(spearman_edge_index)
print(stocks_index_dic)

{'浪潮信息': 0, '中科曙光': 1, '科大讯飞': 2, '昆仑万维': 3, '中际旭创': 4, '景嘉微': 5, '宁德时代': 6, '比亚迪': 7, '隆基绿能': 8, '亿纬锂能': 9, '通威股份': 10, '阳光电源': 11}


In [14]:
"""""转变stock和connect的shape，改变other的index"""

# 循环交易日滚动时间窗口
import os

save_directory='/home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs5days_all1.0'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)
    
for i in range(len(data_trade_time)- window_size + 1):
    
   
    window_data_trade = data_trade.iloc[i+1 : i + window_size+1]  # 提取时间窗口内的数据
    window_spearman= data_spearman.iloc[i : i + window_size]
    # print(window_data_trade)
    # print(window_spearman)
    data1 = names[f"graph_data{i+1}"]  # graph.为tensor
    
    entity_add_dic1={}  # 每个时间窗口内其他类型节点重新排序

    
    # 记录日期范围
    start_date = window_data_trade.iloc[0, 0]  # 第一行的第一列是初始日期
    end_date = window_data_trade.iloc[-1, 0]  # 最后一行的第一列是截止日期
    data1.date_range = (start_date, end_date)  # 自定义添加的异构图属性，不参与图神经网络的计算中
    data1.dates = window_data_trade.iloc[:,0].tolist()
    # print(data1.dates)
    
    # print(data1.dates)


    # node_index
    data1["stock"].node_id = torch.tensor(list(range(12)), dtype=torch.int)
    
    """ node_feature """
    
    # stock_features
    # stocks_feature = window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(1, 10), (13, 22), (25, 34), (37, 46), (49, 58), (61, 70), (73, 82), (85, 94), (97, 106), (109, 118), (121, 130), (133, 142)]], [])].values
    stocks_feature = window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(1, 6), (13, 18), (25, 30), (37, 42), (49, 54), (61, 66), (73, 78), (85, 90), (97, 102), (109, 114), (121, 126), (133, 138)]], [])].values
    stocks_feature_reshaped = stocks_feature.astype(np.float32).reshape(5, 12, 5)   # (天数，股票数，特征)    !!!.astype(np.float32)可以解决object报错
    stocks_feature_reshaped = np.transpose(stocks_feature_reshaped, (1, 0, 2))  # (股票数，天数，特征)
    stocks_feature_reshaped_tensor=torch.tensor(stocks_feature_reshaped, dtype=torch.float)
    stocks_feature_reshaped_tensor=stocks_feature_reshaped_tensor.reshape(12, 25)
    data1["stock"].x = stocks_feature_reshaped_tensor
    
    # connect_feature
    connect_feature = window_data_trade.iloc[:, 7:8].values
    # 扁平化数据：从列表的列表转换为一维数组
    connect_feature_flat = np.array([item for sublist in connect_feature for item in sublist])
    # 转换为整数型，以便用于PyTorch
    connect_feature_flat = connect_feature_flat.astype(np.int32)
    connect_feature_tensor = torch.tensor(connect_feature_flat, dtype=torch.float)
    connect_feature_reshapeed_tensor=connect_feature_tensor.unsqueeze(0)
    data1["connect"].x = connect_feature_reshapeed_tensor  # 表示沪深港的开通状态
    
    # financing & selling feature
    data1["financing"].x = torch.tensor([[1, 0]], dtype=torch.float)
    data1["selling"].x = torch.tensor([[0, 1]], dtype=torch.float)
    
    """ edge (index:(2,n)) """
    # spearman
    data1["stock","spearman","stock"].edge_index = torch.tensor(spearman_edge_index, dtype=torch.long)
    
    spearman_cor = window_spearman.iloc[:,1:].values
    spearman_cor_reshape=spearman_cor.T
    spearman_cor_reshape = torch.tensor(spearman_cor_reshape, dtype=torch.float)
    # 双向边，边特征复制一下
    expanded_edge_attr = torch.cat([spearman_cor_reshape, spearman_cor_reshape], dim=0)
    # print(spearman_feature_reshape.shape)
    # print(spearman_cor)
    # print(spearman_cor.shape)
    # print(spearman_cor_reshape)
    # print(spearman_cor_reshape.shape)
    # print(expanded_edge_attr.shape)
    data1["stock","spearman","stock"].edge_attr = expanded_edge_attr
    
    # connect 沪深港通
    data1["connect", "invest", "stock"].edge_index = torch.tensor([[0,0,0,0,0,0,0,0,0,0,0,0], [0,1,2,3,4,5,6,7,8,9,10,11]], dtype=torch.long)

    connect_flow = window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(6, 7), (18, 19), (30, 31), (42, 43), (54, 55), (66, 67), (78, 79), (90, 91), (102, 103), (114, 115), (126, 127), (138, 139)]], [])].values
    connect_flow_reshape=connect_flow.T
    connect_flow_float = connect_flow_reshape.astype(np.float32)
    data1["connect", "invest", "stock"].edge_attr = torch.tensor(connect_flow_float, dtype=torch.float)
    
    # financing 融资
    data1["financing", "invest", "stock"].edge_index = torch.tensor([[0,0,0,0,0,0,0,0,0,0,0,0], [0,1,2,3,4,5,6,7,8,9,10,11]], dtype=torch.long)
    
    financing_flow=window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(8, 9), (20, 21), (32, 33), (44, 45), (56, 57), (68, 69), (80, 81), (92, 93), (104, 105), (116, 117), (128, 129), (140, 141)]], [])].values
    financing_flow_reshape=financing_flow.T
    financing_flow_float = financing_flow_reshape.astype(np.float32)
    data1["financing", "invest", "stock"].edge_attr = torch.tensor(financing_flow_float, dtype=torch.float)
    
    # selling 融券
    data1["selling", "invest", "stock"].edge_index = torch.tensor([[0,0,0,0,0,0,0,0,0,0,0,0], [0,1,2,3,4,5,6,7,8,9,10,11]], dtype=torch.long)
    
    selling_flow=window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(9, 10), (21, 22), (33, 34), (45, 46), (57, 58), (69, 70), (81, 82), (93, 94), (105, 106), (117, 118), (129, 130), (141, 142)]], [])].values
    selling_flow_reshape=selling_flow.T
    selling_flow_float=selling_flow_reshape.astype(np.float32)
    data1["selling", "invest", "stock"].edge_attr= torch.tensor(selling_flow_float, dtype=torch.float)
    
    # label(对应第六天的实际涨跌)
    label_set = data_trade.iloc[i+window_size+1 : i + window_size+2].iloc[:, sum([list(range(start, end)) for start, end in [(10, 11), (22, 23), (34,35), (46, 47), (58, 59), (70, 71), (82, 83), (94, 95), (106, 107), (118, 119), (130, 131), (142, 143)]], [])].values
    label_set_float=label_set.astype(np.float32)
    data1["stock"].y = torch.tensor(label_set_float, dtype=torch.float)
    
  
    """ 匹配事件 """
    found = False  # 标记是否找到匹配的日期
    
    # 创建空特征、空边及空的边属性,保证该时间窗口内所有店共享索引，
    data1['other'].x = torch.empty((0, 768), dtype=torch.float)
    
    data1['stock', 'relationship', 'stock'].edge_index = torch.tensor([], dtype=torch.long).reshape(2,0)
    data1['stock', 'relationship', 'other'].edge_index = torch.tensor([], dtype=torch.long).reshape(2,0)
    data1['other', 'relationship', 'stock'].edge_index = torch.tensor([], dtype=torch.long).reshape(2,0)
    data1['other', 'relationship', 'other'].edge_index = torch.tensor([], dtype=torch.long).reshape(2,0)

    # 创建初始化的边特征
    data1['stock', 'relationship', 'stock'].edge_attr=torch.empty((0, 768), dtype=torch.float)
    data1['stock', 'relationship', 'other'].edge_attr=torch.empty((0, 768), dtype=torch.float)
    data1['other', 'relationship', 'stock'].edge_attr=torch.empty((0, 768), dtype=torch.float)
    data1['other', 'relationship', 'other'].edge_attr=torch.empty((0, 768), dtype=torch.float)
    for date in data1.dates: 
        print(date)
        # entity_add_dic1={}
        # entity_add_dic2={}
        # entity_add_dic3={}
        # entity_add_dic4={}
        
        if pd.Timestamp(date) in event_unique_dates_list:  # 先转化为时间戳格式，如果有匹配的日期才会生成时间边索引及对应边特征
                   
            # 筛选 data_event 中日期匹配的行
            matched_rows = data_event[data_event['事件截止时间（数值）'].isin([date])].index
            matched_rows_data = data_event.loc[matched_rows].iloc[:, 2:5]
            # print(matched_rows_data)
            found=True
            
            # 创建元组填充的列表，元组的操作和字典很像，只不过元组的内容一经制定无法修改
            event_tuples = [] 
            for idx, row in matched_rows_data.iterrows():  # row是pd.series
                event_tuples.append((row['entity1'], row['relationship'], row['entity2']))
                
            for item in event_tuples:
                ## print(item)    
                
                # 前两个都是针对 ['stock', 'relationship', 'stock']
                if item[0] in stocks_index_dic and item[2] in stocks_index_dic:  # 单向边  
                                     
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[2]))],dim=0).unsqueeze(1)
                    data1['stock', 'relationship', 'stock'].edge_index = torch.cat([data1['stock', 'relationship', 'stock'].edge_index, new_edge_stack], dim=1)
                    
                    data1['stock', 'relationship', 'stock'].edge_attr= torch.cat([data1['stock', 'relationship', 'stock'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                    
                    
                elif item[0] in stocks_index_dic and pd.isna(item[2]):  # 无向边                    
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[0]))],dim=0).unsqueeze(1)
                    data1['stock', 'relationship', 'stock'].edge_index = torch.cat([data1['stock', 'relationship', 'stock'].edge_index, new_edge_stack], dim=1)
                    
                    data1['stock', 'relationship', 'stock'].edge_attr= torch.cat([data1['stock', 'relationship', 'stock'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                
                
                elif item[0] in stocks_index_dic and item[2] not in stocks_index_dic and not pd.isna(item[2]):
                    if item[2] not in entity_add_dic1:
                        data1['other'].x= torch.cat((data1['other'].x, extract_last_layer_hidden(model,tokenizer,item[2])), dim=0)  # shape(N,768)
                    
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[2]))],dim=0).unsqueeze(1)                    
                    data1['stock', 'relationship', 'other'].edge_index = torch.cat([data1['stock', 'relationship', 'other'].edge_index, new_edge_stack], dim=1)

                    data1['stock', 'relationship', 'other'].edge_attr= torch.cat([data1['stock', 'relationship', 'other'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)

                       
                elif item[0] not in stocks_index_dic and not pd.isna(item[0]) and item[2] in stocks_index_dic:      
                    if item[0] not in entity_add_dic1:
                        data1['other'].x= torch.cat((data1['other'].x, extract_last_layer_hidden(model,tokenizer,item[0])), dim=0)       
                          
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[2]))],dim=0).unsqueeze(1)
                    data1['other', 'relationship', 'stock'].edge_index = torch.cat([data1['other', 'relationship', 'stock'].edge_index, new_edge_stack], dim=1)
                    
                    data1['other', 'relationship', 'stock'].edge_attr= torch.cat([data1['other', 'relationship', 'stock'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                    
                    
                elif item[0] not in stocks_index_dic and not pd.isna(item[0]) and item[2] not in stocks_index_dic and not pd.isna(item[2]):  # 单向边   
                    if item[0] not in entity_add_dic1:
                        # 如果 item[0] 不在 entity_add_dic4 中，则添加节点特征
                        data1['other'].x = torch.cat((data1['other'].x, extract_last_layer_hidden(model, tokenizer, item[0])), dim=0)
                    
                    if item[2] not in entity_add_dic1:
                        # 如果 item[2] 不在 entity_add_dic4 中，则添加节点特征
                        data1['other'].x = torch.cat((data1['other'].x, extract_last_layer_hidden(model, tokenizer, item[2])), dim=0)        
                                 
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[2]))],dim=0).unsqueeze(1)
                    data1['other', 'relationship', 'other'].edge_index = torch.cat([data1['other', 'relationship', 'other'].edge_index, new_edge_stack], dim=1)
                    
                    data1['other', 'relationship', 'other'].edge_attr= torch.cat([data1['other', 'relationship', 'other'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                    
                   
                    
                elif item[0] not in stocks_index_dic and not pd.isna(item[0]) and pd.isna(item[2]):  # 无向边   
                    if item[0] not in entity_add_dic1:
                        data1['other'].x= torch.cat((data1['other'].x, extract_last_layer_hidden(model,tokenizer,item[0])), dim=0)       
                                         
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[0]))],dim=0).unsqueeze(1)
                    data1['other', 'relationship', 'other'].edge_index = torch.cat([data1['other', 'relationship', 'other'].edge_index, new_edge_stack], dim=1)
                    
                    data1['other', 'relationship', 'other'].edge_attr= torch.cat([data1['other', 'relationship', 'other'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                    
                                         
        else:
            print("没有找到匹配的日期。")
            
    # print("other节点数量", data1['other'].x.size(0))
    # print("关系边种other的index最大值",  data1['stock', 'relationship', 'other'].edge_index[1].max().item())      
            
    file_name = f"graph_{i+1}.pt"
    file_path = os.path.join(save_directory, file_name)
        
    torch.save(data1, file_path)
    print(f"Saved graph to {file_path}")        
            
            
            
    # print(stocks_feature_reshaped_tensor.shape)
    # print(data1)
    # print(connect_feature_reshapeed_tensor.shape)
    # print(data1["connect"].x)
    # print(data1["financing"].x)
    


ValueError: cannot reshape array of size 60 into shape (5,12,5)

In [16]:
"""""转变stock和connect的shape，改变other的index"""

# 循环交易日滚动时间窗口
import os

save_directory='/home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)
    
for i in range(len(data_trade_time)- window_size + 1):
    
   
    window_data_trade = data_trade.iloc[i+1 : i + window_size+1]  # 提取时间窗口内的数据
    window_spearman= data_spearman.iloc[i : i + window_size]
    # print(window_data_trade)
    # print(window_spearman)
    data1 = names[f"graph_data{i+1}"]  # graph.为tensor
    
    entity_add_dic1={}  # 每个时间窗口内其他类型节点重新排序

    
    # 记录日期范围
    start_date = window_data_trade.iloc[0, 0]  # 第一行的第一列是初始日期
    end_date = window_data_trade.iloc[-1, 0]  # 最后一行的第一列是截止日期
    data1.date_range = (start_date, end_date)  # 自定义添加的异构图属性，不参与图神经网络的计算中
    data1.dates = window_data_trade.iloc[:,0].tolist()
    # print(data1.dates)
    
    # print(data1.dates)


    # node_index
    data1["stock"].node_id = torch.tensor(list(range(12)), dtype=torch.int)
    
    """ node_feature """
    
    # stock_features
    # stocks_feature = window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(1, 10), (13, 22), (25, 34), (37, 46), (49, 58), (61, 70), (73, 82), (85, 94), (97, 106), (109, 118), (121, 130), (133, 142)]], [])].values
    stocks_feature = window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(1, 6), (13, 18), (25, 30), (37, 42), (49, 54), (61, 66), (73, 78), (85, 90), (97, 102), (109, 114), (121, 126), (133, 138)]], [])].values
    stocks_feature_reshaped = stocks_feature.astype(np.float32).reshape(1, 12, 5)   # (天数，股票数，特征)    !!!.astype(np.float32)可以解决object报错
    stocks_feature_reshaped = np.transpose(stocks_feature_reshaped, (1, 0, 2))  # (股票数，天数，特征)
    stocks_feature_reshaped_tensor=torch.tensor(stocks_feature_reshaped, dtype=torch.float)
    stocks_feature_reshaped_tensor=stocks_feature_reshaped_tensor.reshape(12, 5)
    data1["stock"].x = stocks_feature_reshaped_tensor
    
    # connect_feature
    connect_feature = window_data_trade.iloc[:, 7:8].values
    # 扁平化数据：从列表的列表转换为一维数组
    connect_feature_flat = np.array([item for sublist in connect_feature for item in sublist])
    # 转换为整数型，以便用于PyTorch
    connect_feature_flat = connect_feature_flat.astype(np.int32)
    connect_feature_tensor = torch.tensor(connect_feature_flat, dtype=torch.float)
    connect_feature_reshapeed_tensor=connect_feature_tensor.unsqueeze(0)
    data1["connect"].x = connect_feature_reshapeed_tensor  # 表示沪深港的开通状态
    
    # financing & selling feature
    data1["financing"].x = torch.tensor([[1, 0]], dtype=torch.float)
    data1["selling"].x = torch.tensor([[0, 1]], dtype=torch.float)
    
    """ edge (index:(2,n)) """
    # spearman
    data1["stock","spearman","stock"].edge_index = torch.tensor(spearman_edge_index, dtype=torch.long)
    
    spearman_cor = window_spearman.iloc[:,1:].values
    spearman_cor_reshape=spearman_cor.T
    spearman_cor_reshape = torch.tensor(spearman_cor_reshape, dtype=torch.float)
    # 双向边，边特征复制一下
    expanded_edge_attr = torch.cat([spearman_cor_reshape, spearman_cor_reshape], dim=0)
    # print(spearman_feature_reshape.shape)
    # print(spearman_cor)
    # print(spearman_cor.shape)
    # print(spearman_cor_reshape)
    # print(spearman_cor_reshape.shape)
    # print(expanded_edge_attr.shape)
    data1["stock","spearman","stock"].edge_attr = expanded_edge_attr
    
    # connect 沪深港通
    data1["connect", "invest", "stock"].edge_index = torch.tensor([[0,0,0,0,0,0,0,0,0,0,0,0], [0,1,2,3,4,5,6,7,8,9,10,11]], dtype=torch.long)

    connect_flow = window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(6, 7), (18, 19), (30, 31), (42, 43), (54, 55), (66, 67), (78, 79), (90, 91), (102, 103), (114, 115), (126, 127), (138, 139)]], [])].values
    connect_flow_reshape=connect_flow.T
    connect_flow_float = connect_flow_reshape.astype(np.float32)
    data1["connect", "invest", "stock"].edge_attr = torch.tensor(connect_flow_float, dtype=torch.float)
    
    # financing 融资
    data1["financing", "invest", "stock"].edge_index = torch.tensor([[0,0,0,0,0,0,0,0,0,0,0,0], [0,1,2,3,4,5,6,7,8,9,10,11]], dtype=torch.long)
    
    financing_flow=window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(8, 9), (20, 21), (32, 33), (44, 45), (56, 57), (68, 69), (80, 81), (92, 93), (104, 105), (116, 117), (128, 129), (140, 141)]], [])].values
    financing_flow_reshape=financing_flow.T
    financing_flow_float = financing_flow_reshape.astype(np.float32)
    data1["financing", "invest", "stock"].edge_attr = torch.tensor(financing_flow_float, dtype=torch.float)
    
    # selling 融券
    data1["selling", "invest", "stock"].edge_index = torch.tensor([[0,0,0,0,0,0,0,0,0,0,0,0], [0,1,2,3,4,5,6,7,8,9,10,11]], dtype=torch.long)
    
    selling_flow=window_data_trade.iloc[:, sum([list(range(start, end)) for start, end in [(9, 10), (21, 22), (33, 34), (45, 46), (57, 58), (69, 70), (81, 82), (93, 94), (105, 106), (117, 118), (129, 130), (141, 142)]], [])].values
    selling_flow_reshape=selling_flow.T
    selling_flow_float=selling_flow_reshape.astype(np.float32)
    data1["selling", "invest", "stock"].edge_attr= torch.tensor(selling_flow_float, dtype=torch.float)
    
    # label(对应第六天的实际涨跌)
    label_set = data_trade.iloc[i+window_size+1 : i + window_size+2].iloc[:, sum([list(range(start, end)) for start, end in [(10, 11), (22, 23), (34,35), (46, 47), (58, 59), (70, 71), (82, 83), (94, 95), (106, 107), (118, 119), (130, 131), (142, 143)]], [])].values
    label_set_float=label_set.astype(np.float32)
    data1["stock"].y = torch.tensor(label_set_float, dtype=torch.float)
    
  
    """ 匹配事件 """
    found = False  # 标记是否找到匹配的日期
    
    # 创建空特征、空边及空的边属性,保证该时间窗口内所有店共享索引，
    data1['other'].x = torch.empty((0, 768), dtype=torch.float)
    
    data1['stock', 'relationship', 'stock'].edge_index = torch.tensor([], dtype=torch.long).reshape(2,0)
    data1['stock', 'relationship', 'other'].edge_index = torch.tensor([], dtype=torch.long).reshape(2,0)
    data1['other', 'relationship', 'stock'].edge_index = torch.tensor([], dtype=torch.long).reshape(2,0)
    data1['other', 'relationship', 'other'].edge_index = torch.tensor([], dtype=torch.long).reshape(2,0)

    # 创建初始化的边特征
    data1['stock', 'relationship', 'stock'].edge_attr=torch.empty((0, 768), dtype=torch.float)
    data1['stock', 'relationship', 'other'].edge_attr=torch.empty((0, 768), dtype=torch.float)
    data1['other', 'relationship', 'stock'].edge_attr=torch.empty((0, 768), dtype=torch.float)
    data1['other', 'relationship', 'other'].edge_attr=torch.empty((0, 768), dtype=torch.float)
    for date in data1.dates: 
        print(date)
        # entity_add_dic1={}
        # entity_add_dic2={}
        # entity_add_dic3={}
        # entity_add_dic4={}
        
        if pd.Timestamp(date) in event_unique_dates_list:  # 先转化为时间戳格式，如果有匹配的日期才会生成时间边索引及对应边特征
                   
            # 筛选 data_event 中日期匹配的行
            matched_rows = data_event[data_event['事件截止时间（数值）'].isin([date])].index
            matched_rows_data = data_event.loc[matched_rows].iloc[:, 2:5]
            # print(matched_rows_data)
            found=True
            
            # 创建元组填充的列表，元组的操作和字典很像，只不过元组的内容一经制定无法修改
            event_tuples = [] 
            for idx, row in matched_rows_data.iterrows():  # row是pd.series
                event_tuples.append((row['entity1'], row['relationship'], row['entity2']))
                
            for item in event_tuples:
                ## print(item)    
                
                # 前两个都是针对 ['stock', 'relationship', 'stock']
                if item[0] in stocks_index_dic and item[2] in stocks_index_dic:  # 单向边  
                                     
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[2]))],dim=0).unsqueeze(1)
                    data1['stock', 'relationship', 'stock'].edge_index = torch.cat([data1['stock', 'relationship', 'stock'].edge_index, new_edge_stack], dim=1)
                    
                    data1['stock', 'relationship', 'stock'].edge_attr= torch.cat([data1['stock', 'relationship', 'stock'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                    
                    
                elif item[0] in stocks_index_dic and pd.isna(item[2]):  # 无向边                    
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[0]))],dim=0).unsqueeze(1)
                    data1['stock', 'relationship', 'stock'].edge_index = torch.cat([data1['stock', 'relationship', 'stock'].edge_index, new_edge_stack], dim=1)
                    
                    data1['stock', 'relationship', 'stock'].edge_attr= torch.cat([data1['stock', 'relationship', 'stock'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                
                
                elif item[0] in stocks_index_dic and item[2] not in stocks_index_dic and not pd.isna(item[2]):
                    if item[2] not in entity_add_dic1:
                        data1['other'].x= torch.cat((data1['other'].x, extract_last_layer_hidden(model,tokenizer,item[2])), dim=0)  # shape(N,768)
                    
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[2]))],dim=0).unsqueeze(1)                    
                    data1['stock', 'relationship', 'other'].edge_index = torch.cat([data1['stock', 'relationship', 'other'].edge_index, new_edge_stack], dim=1)

                    data1['stock', 'relationship', 'other'].edge_attr= torch.cat([data1['stock', 'relationship', 'other'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)

                       
                elif item[0] not in stocks_index_dic and not pd.isna(item[0]) and item[2] in stocks_index_dic:      
                    if item[0] not in entity_add_dic1:
                        data1['other'].x= torch.cat((data1['other'].x, extract_last_layer_hidden(model,tokenizer,item[0])), dim=0)       
                          
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[2]))],dim=0).unsqueeze(1)
                    data1['other', 'relationship', 'stock'].edge_index = torch.cat([data1['other', 'relationship', 'stock'].edge_index, new_edge_stack], dim=1)
                    
                    data1['other', 'relationship', 'stock'].edge_attr= torch.cat([data1['other', 'relationship', 'stock'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                    
                    
                elif item[0] not in stocks_index_dic and not pd.isna(item[0]) and item[2] not in stocks_index_dic and not pd.isna(item[2]):  # 单向边   
                    if item[0] not in entity_add_dic1:
                        # 如果 item[0] 不在 entity_add_dic4 中，则添加节点特征
                        data1['other'].x = torch.cat((data1['other'].x, extract_last_layer_hidden(model, tokenizer, item[0])), dim=0)
                    
                    if item[2] not in entity_add_dic1:
                        # 如果 item[2] 不在 entity_add_dic4 中，则添加节点特征
                        data1['other'].x = torch.cat((data1['other'].x, extract_last_layer_hidden(model, tokenizer, item[2])), dim=0)        
                                 
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[2]))],dim=0).unsqueeze(1)
                    data1['other', 'relationship', 'other'].edge_index = torch.cat([data1['other', 'relationship', 'other'].edge_index, new_edge_stack], dim=1)
                    
                    data1['other', 'relationship', 'other'].edge_attr= torch.cat([data1['other', 'relationship', 'other'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                    
                   
                    
                elif item[0] not in stocks_index_dic and not pd.isna(item[0]) and pd.isna(item[2]):  # 无向边   
                    if item[0] not in entity_add_dic1:
                        data1['other'].x= torch.cat((data1['other'].x, extract_last_layer_hidden(model,tokenizer,item[0])), dim=0)       
                                         
                    new_edge_stack=torch.stack([torch.tensor(entity_process1(item[0])),torch.tensor(entity_process1(item[0]))],dim=0).unsqueeze(1)
                    data1['other', 'relationship', 'other'].edge_index = torch.cat([data1['other', 'relationship', 'other'].edge_index, new_edge_stack], dim=1)
                    
                    data1['other', 'relationship', 'other'].edge_attr= torch.cat([data1['other', 'relationship', 'other'].edge_attr, extract_last_layer_hidden(model, tokenizer, item[1])], dim=0)
                    
                                         
        else:
            print("没有找到匹配的日期。")
            
    # print("other节点数量", data1['other'].x.size(0))
    # print("关系边种other的index最大值",  data1['stock', 'relationship', 'other'].edge_index[1].max().item())      
            
    file_name = f"graph_{i+1}.pt"
    file_path = os.path.join(save_directory, file_name)
        
    torch.save(data1, file_path)
    print(f"Saved graph to {file_path}")        
            
            
            
    # print(stocks_feature_reshaped_tensor.shape)
    # print(data1)
    # print(connect_feature_reshapeed_tensor.shape)
    # print(data1["connect"].x)
    # print(data1["financing"].x)
    


2021-01-11
Saved graph to /home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0/graph_1.pt
2021-01-12
Saved graph to /home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0/graph_2.pt
2021-01-13
Saved graph to /home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0/graph_3.pt
2021-01-14
Saved graph to /home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0/graph_4.pt
2021-01-15
Saved graph to /home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0/graph_5.pt
2021-01-18
Saved graph to /home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0/graph_6.pt
2021-01-19
Saved graph to /home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0/graph_7.pt
2021-01-20
Saved graph to /home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0/graph_8.pt
2021-01-21
Saved graph to /home/QFRC/ZQS/Projects/Graph/Scripts_demo/Graphs_demos/Graphs1days_all1.0/gra